# Analysis Template

A template with the tools to get started pulling data from a local db.

In [3]:
import os
import asyncio
import requests
import pandas as pd
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
import sqlite3
import plotly.express as px

pd.options.display.float_format = '{:.4f}'.format

In [4]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")


In [5]:
df_trades = pd.read_sql_query('SELECT * FROM trades', con)
df_trades['date'] = pd.to_datetime(df_trades['timestamp'], unit='s')
df_trades['day'] = df_trades['date'].dt.date
df_trades['volume'] = abs(df_trades['size'] * df_trades['price'])

df_trades.head(5)

,id,account,abstractAccount,accountType,timestamp,asset,marketKey,orderType,price,margin,positionId,feesPaid,pnl,size,positionSize,positionClosed,trackingCode,date,day,volume
0,0x00000bc7dff9d0236799f8505937d39cde71e13ccf79...,0xac9c065b1a84a486628fd20efc9426e435ba7cc2,0xac9c065b1a84a486628fd20efc9426e435ba7cc2,isolated_margin,1680895361,SOL,sSOLPERP,DelayedOffchain,20.4749,104.2841,0x0ea09d97b4084d859328ec4bf8ebcf9ecca26f1d-0x3bd,2.4886,-7.2137,29.8275,0.0000,1,KWENTA,2023-04-07 19:22:41,2023-04-07,610.7163
1,0x0000709c04c4a1f048371ca2e2641740b161f24db3da...,0x02bee0a17903cc1f2f48b4514ec23aae547fe4a3,0x02bee0a17903cc1f2f48b4514ec23aae547fe4a3,isolated_margin,1678414645,OP,sOPPERP,DelayedOffchain,2.2136,358.1664,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x654,17.1125,-34.9980,-4551.4423,0.0000,1,KWENTA,2023-03-10 02:17:25,2023-03-10,10074.9771
2,0x000080a930a760cdbe740044a11622e717257fbe7383...,0xbf975b1548f793f3bcc2ce84dcf8e78ece793914,0xbf975b1548f793f3bcc2ce84dcf8e78ece793914,isolated_margin,1678799771,OP,sOPPERP,DelayedOffchain,2.8104,43622.2839,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x4f7,171.4718,23634.8226,-40201.7427,0.0000,1,KWENTA,2023-03-14 13:16:11,2023-03-14,112981.1980
3,0x000092e1e5d70976b281207016b94f03d7b5c7eca12c...,0x6662304e3198c8d54530fbdf02a869fa84537639,0x6662304e3198c8d54530fbdf02a869fa84537639,isolated_margin,1678622207,OP,sOPPERP,DelayedOffchain,1.8834,8878.5906,0x442b69937a0daf9d46439a71567fabe6cb69fbaf-0x70a,14.5350,0.0000,-6655.5716,-117749.8870,0,KWENTA,2023-03-12 11:56:47,2023-03-12,12535.0120
4,0x0000a7f953004419f63ce2dbc907c0f72a325efc6f2a...,0x01c8c06eb7a35f978f5629461552f57bc9783acd,0x01c8c06eb7a35f978f5629461552f57bc9783acd,isolated_margin,1680176858,sETH,sETHPERP,DelayedOffchain,1816.3878,232426.6330,0x2b3bb4c683bfc5239b029131eef3b1d214478d93-0x19e8,92.8194,0.0000,-100.0000,-450.0000,0,KWENTA,2023-03-30 11:47:38,2023-03-30,181638.7800


In [13]:
df_volume = df_trades.groupby('day').agg({'volume': 'sum'}).reset_index()
df_volume['volume_trailing'] = df_volume['volume'].rolling(7).sum()

In [14]:
df_volume

,day,volume,volume_trailing
0,2022-12-20,32.0280,NaN
1,2022-12-21,185700.7861,NaN
2,2022-12-22,364307.4995,NaN
3,2022-12-23,854106.0617,NaN
4,2022-12-24,1868442.7050,NaN
...,...,...,...
136,2023-05-05,220884420.1789,840887068.9137
137,2023-05-06,251048403.4402,1049354441.7716
138,2023-05-07,140798027.9264,1122233331.6290
139,2023-05-08,177217080.0779,1152871894.3987


In [19]:

fig = px.line(
    df_volume,
    x='day',
    y='volume_trailing',
    title='7 Day Trailing Volume',
)

fig.show()